# 0.0. Info Table

Monster Hunter 2 Dos Database App

    1. Lista de Quests separadas por Ranks.

In [19]:
import re
import sqlite3
import pandas as pd

from sqlalchemy import create_engine

# 1.0. Kitchen Combinations Data

## 1.1. Clean Data Function

In [47]:
def clean_data( df ):
    # Filter Columns and Rename
    df.columns = ['no', 'first_food', 'second_food', 'recipe', 'health', 'stamina', 'attack', 'defense']
    df = df.drop( columns=['no'], axis=1 )
    df = df[~df['first_food'].str.contains('Foods')]

    # Individual Columns
    df['first_food']  = df['first_food'].str.replace('\xa0', ' ')
    df['second_food'] = df['second_food'].str.replace('\xa0', ' ')

    df['health']  = df['health'].fillna( 'Health：+0' )
    df['stamina'] = df['stamina'].fillna( 'Stamina：+0' )
    df['attack']  = df['attack'].fillna( 'Atk Up：+0' )
    df['defense'] = df['defense'].fillna( 'Def Up：+0' )

    df['health']  = df['health'].str.extract('(\-?\+?\d+)')[0]
    df['stamina'] = df['stamina'].str.extract('(\-?\+?\d+)')[0]
    df['attack']  = df['attack'].str.extract('(\-?\+?\d+)')[0]
    df['defense'] = df['defense'].str.extract('(\-?\+?\d+)')[0]

    df = df.reset_index( drop=True )
    
    return df

## 1.2. Load and Clean Recipes Data

In [50]:
df_winter  = pd.read_excel( '../pdf_excel/winter.xlsx' )
df_summer  = pd.read_excel( '../pdf_excel/summer.xlsx' )
df_breeder = pd.read_excel( '../pdf_excel/breeder.xlsx' )

df_winter  = clean_data( df=df_winter )
df_summer  = clean_data( df=df_summer )
df_breeder = clean_data( df=df_breeder )

## 1.3. Kitchen Recipes Storange

In [57]:
df_winter.to_csv( '../data/recipes_winter.csv' )
df_summer.to_csv( '../data/recipes_summer.csv' )
df_breeder.to_csv( '../data/recipes_breeder.csv' )

list_seasons = ['winter', 'summer', 'breeder']

In [67]:
con = sqlite3.connect( '../sql_databases/recipes.sqlite' )
c   = con.cursor()

for i in range( len( list_seasons) ):
    query_crate_table = '''
        CREATE TABLE ''' + list_seasons[i] + ''' (
            first_food     TEXT,
            second_food    TEXT,
            recipe         TEXT,
            health         TEXT,
            stamina        TEXT,
            attack         TEXT,
            defense        TEXT );'''
    
    c.execute(query_crate_table)
    con.commit()

winter
summer
breeder


In [69]:
db  = create_engine( 'sqlite:///../sql_databases/recipes.sqlite' )
con = db.connect()

df_winter.to_sql(  'winter',  con=con, if_exists='append', index=False )
df_summer.to_sql(  'summer',  con=con, if_exists='append', index=False )
df_breeder.to_sql( 'breeder', con=con, if_exists='append', index=False )